In [2]:
import warnings

warnings.filterwarnings('ignore')

In [1]:
# use gnu-grep with perl regexp & gnu-sed

!ggrep -P '\tstart_codon\t.*\t\+\t.*transcript_support_level\s"1"' ../data/binfo1-datapack1/gencode.gtf | \
    gsed -e 's/\t[^\t]*transcript_id\s"\([^"]*\)".*$/\t\1/g' > ../data/binfo1-datapack1/gencode-start.gtf

In [3]:
!head ../data/binfo1-datapack1/gencode-start.gtf

chr1	HAVANA	start_codon	4878137	4878139	.	+	0	ENSMUST00000134384.8
chr1	HAVANA	start_codon	4878137	4878139	.	+	0	ENSMUST00000027036.11
chr1	HAVANA	start_codon	4878137	4878139	.	+	0	ENSMUST00000150971.8
chr1	HAVANA	start_codon	4928137	4928139	.	+	0	ENSMUST00000081551.14
chr1	HAVANA	start_codon	5154674	5154676	.	+	0	ENSMUST00000044369.13
chr1	HAVANA	start_codon	5659272	5659274	.	+	0	ENSMUST00000160777.8
chr1	HAVANA	start_codon	5659272	5659274	.	+	0	ENSMUST00000027038.11
chr1	HAVANA	start_codon	6300227	6300229	.	+	0	ENSMUST00000027040.13
chr1	HAVANA	start_codon	6429555	6429557	.	+	0	ENSMUST00000133144.4
chr1	HAVANA	start_codon	6839122	6839124	.	+	0	ENSMUST00000140079.8


In [4]:
!ggrep -P '\texon\t.*\t\+\t' ../data/binfo1-datapack1/gencode.gtf | \
    gsed -e 's/\t[^\t]*transcript_id\s"\([^"]*\)".*$/\t\1/g' > ../data/binfo1-datapack1/gencode-plusexon.gtf

In [5]:
!head ../data/binfo1-datapack1/gencode-plusexon.gtf

chr1	HAVANA	exon	3143476	3144545	.	+	.	ENSMUST00000193812.2
chr1	ENSEMBL	exon	3172239	3172348	.	+	.	ENSMUST00000082908.3
chr1	HAVANA	exon	3322980	3323459	.	+	.	ENSMUST00000192857.2
chr1	HAVANA	exon	3536810	3536910	.	+	.	ENSMUST00000161581.2
chr1	HAVANA	exon	3583628	3583776	.	+	.	ENSMUST00000161581.2
chr1	HAVANA	exon	3602018	3602943	.	+	.	ENSMUST00000192183.2
chr1	HAVANA	exon	3750378	3752011	.	+	.	ENSMUST00000193244.2
chr1	HAVANA	exon	3822233	3824583	.	+	.	ENSMUST00000194454.2
chr1	HAVANA	exon	4566774	4569601	.	+	.	ENSMUST00000193450.2
chr1	HAVANA	exon	4567697	4567877	.	+	.	ENSMUST00000194935.2


In [6]:
# sort -k option explanation : https://sidepower.tistory.com/24

!bedtools intersect -a ../data/binfo1-datapack1/gencode-start.gtf -b ../data/binfo1-datapack1/gencode-plusexon.gtf -wa -wb | \
    awk -F '\t' -v OFS='\t' '$9 == $18 {print $10, $13-1, $14, $18, $4-1, $16;}' | \
        sort -k1,1 -k2,3n -k4,4 > ../data/binfo1-datapack1/gencode-exons-containing-startcodon.bed

In [11]:
# 1st column : which chromosome?
# 2nd column : location of start of sequence (gene/transcript/etc)
# 3rd column : location of end of sequence (gene/transcript/etc)
# 4th column : transcript id
# 5th column : location of start codon
# 6th column : which strand? (+ for forward, - for reverse)
!head ../data/binfo1-datapack1/gencode-exons-containing-startcodon.bed; \
    tail ../data/binfo1-datapack1/gencode-exons-containing-startcodon.bed

chr1	4878010	4878205	ENSMUST00000134384.8	4878136	+
chr1	4878045	4878205	ENSMUST00000027036.11	4878136	+
chr1	4878052	4878205	ENSMUST00000150971.8	4878136	+
chr1	4928036	4928199	ENSMUST00000081551.14	4928136	+
chr1	5154639	5154786	ENSMUST00000044369.13	5154673	+
chr1	5659227	5659528	ENSMUST00000027038.11	5659271	+
chr1	5659257	5659528	ENSMUST00000160777.8	5659271	+
chr1	6300182	6300297	ENSMUST00000027040.13	6300226	+
chr1	6429441	6429738	ENSMUST00000133144.4	6429554	+
chr1	6839121	6839176	ENSMUST00000043578.13	6839121	+
chrY	80939672	80939804	ENSMUST00000185340.2	80939686	+
chrY	81470683	81470815	ENSMUST00000187135.2	81470697	+
chrY	82237903	82238035	ENSMUST00000185636.2	82237917	+
chrY	83043623	83043755	ENSMUST00000187165.2	83043637	+
chrY	84109956	84110088	ENSMUST00000185776.7	84109970	+
chrY	84759361	84759493	ENSMUST00000186110.2	84759375	+
chrY	86074448	86074580	ENSMUST00000188754.2	86074462	+
chrY	87129500	87129632	ENSMUST00000189543.7	87129514	+
chrY	87563647	87563779	ENSMUST0000

In [5]:
# samtools view options
# -H : output headers only
# -F : flag option
# -b : output file format is 'bam'
# -o : designate output file name

# bioawk options
# -c : specify input format

!(samtools view -H ./../data/binfo1-datapack1/RPF-siLuc.bam; \
    samtools view -F20 ./../data/binfo1-datapack1/RPF-siLuc.bam | \
        bioawk -c sam '{if (length($seq) >= 25) print $0}') | \
            samtools view -b -o ./../data/binfo1-datapack1/filtered-RPF-siLuc.bam

/bin/sh: bioawk: command not found
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


In [3]:
!ls

CoLab_TermProj_2022-2.ipynb prj2.ipynb
